# VISA CASE STUDY

### Problem Statement:
<b>How would you assess the impact of weather on consumer spend last winter?</b>

The following Notebook is an attempt to acheive the following:<br>
1. Frame the above problem coherently
2. Use Analyses techniques on the data
3. Leverage the results and findings to suggest possible solutions/insights
4. Discuss potential limitations of the approach